In [ ]:
!nvidia-smi

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import warnings
warnings.filterwarnings("ignore")

from Parse_TFrecords import *
from define_model import *
from load_label import *
from utilities import *
import gc
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

import numpy as np
from sklearn.metrics import classification_report, average_precision_score

gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
    
print(gpus)

In [ ]:
BATCH_SIZE = 32
dataset = 'mimic'

record_file_test = 'tfrecords/copd_{a}_test.tfrecords'.format(a=dataset)
test_dataset = (tf.data.TFRecordDataset(
    record_file_test, buffer_size=BATCH_SIZE, compression_type=None, num_parallel_reads=32)
.map(parse_TFrecord_test)
.batch(BATCH_SIZE))

y_test, y_demo = get_data_label(dataset=dataset, split='test', return_demo=True)

In [ ]:
archis = ['DenseNet121', 'ResNet50V2', 'MobileNetV2', 'Xception']

preds = []
for archi in archis:
    print(archi)
    checkpoint_filepath = 'checkpoints_merged/checkpoint_BCE_{i}'.format(i=archi)
    model = define_model(archi)
    model.load_weights(checkpoint_filepath)
    
    y_pred = model.predict(test_dataset)
    
    test_CI(y_pred, y_test)

    print(average_precision_score(y_test, y_pred, average=None))

    thresh = get_thresh(y_test, y_pred, 'Youden')
    print('thresh:', thresh)

    print(classification_report(y_test, np.where(y_pred >= thresh, 1, 0)))


    preds.append(y_pred)

preds = np.array(preds)

y_preds_ave = 0
for i in range(4):
    y_preds_ave += preds[i]

y_preds_ave = y_preds_ave/4

test_CI(y_preds_ave, y_test)

print(average_precision_score(y_test, y_preds_ave, average=None))

thresh = get_thresh(y_test, y_preds_ave, 'Youden')
print('thresh:', thresh)

print(classification_report(y_test, np.where(y_preds_ave >= thresh, 1, 0)))

gc.collect()

## Race

In [ ]:
race_list = ['White', 'Black', 'Latino', 'Others', 'Asian']

for race_num, race in enumerate(race_list):
    print(race)
    
    idx = []
    for i, l in enumerate(y_demo):
        if (l['Race']==race_num):
            idx.append(i)

    test_CI(y_preds_ave[idx], y_test[idx])

    print(average_precision_score(y_test[idx], y_preds_ave[idx], average=None))

    thresh = get_thresh(y_test[idx], y_preds_ave[idx], 'Youden')
    print('thresh:', thresh)

    print(classification_report(y_test[idx], np.where(y_preds_ave[idx] >= thresh, 1, 0)))

    gc.collect()

## Gender

In [ ]:
gender_list = ['Female', 'Male']

for gender_num, gender in enumerate(gender_list):
    print(gender)
    
    idx = []
    for i, l in enumerate(y_demo):
        if (l['Gender']==gender_num):
            idx.append(i)

    test_CI(y_preds_ave[idx], y_test[idx])

    print(average_precision_score(y_test[idx], y_preds_ave[idx], average=None))

    thresh = get_thresh(y_test[idx], y_preds_ave[idx], 'Youden')
    print('thresh:', thresh)

    print(classification_report(y_test[idx], np.where(y_preds_ave[idx] >= thresh, 1, 0)))

    gc.collect()

## Age

In [ ]:
age_list = ['0-40', '40-60', '60-80', '80-']

for age_num, age in enumerate(age_list):
    print(age)
    
    idx = []
    for i, l in enumerate(y_demo):
        if (l['Age']==age_num):
            idx.append(i)

    test_CI(y_preds_ave[idx], y_test[idx])

    print(average_precision_score(y_test[idx], y_preds_ave[idx], average=None))

    thresh = get_thresh(y_test[idx], y_preds_ave[idx], 'Youden')
    print('thresh:', thresh)

    print(classification_report(y_test[idx], np.where(y_preds_ave[idx] >= thresh, 1, 0)))

    gc.collect()